# Notebook: Mô hình tóm tắt văn bản (Summarization)

In [ ]:
# Import các thư viện cần thiết

import json
import numpy as np

from scipy import spatial
from operator import itemgetter

In [ ]:
pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 19.2 MB/s eta 0:00:00


In [ ]:
! pip install transformers
! pip install datasets
! pip install evaluate
! pip install rouge_score

import evaluate
from datasets import load_dataset

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 40.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 37.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 28.0 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.18.0
    Uninstalling huggingface-hub-0.18.0:
      Successfully uninstalled huggingface-hub-0.18.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24932 sha256=a4ba6732096f8ec5dccbb9b68f

### **Đọc - Hiểu cấu trúc dữ liệu đầu vào**
Mô tả cấu trúc bộ dữ liệu BillSum

Bộ dữ liệu tóm tắt các dự luật của Quốc hội Hoa Kỳ và tiểu bang California gồm các thuộc tính:
* text: nôi dung của văn bản.
* summary: tóm tắt của văn bản.
* title: tiêu đề của văn bản(có thể không có)
* text_len: số ký tự trong văn bản.
* sum_len: số ký tự tóm tắt.



In [ ]:
# Download bộ dữ liệu billsum

billsum = load_dataset("billsum", split="test")

# Chia bộ dữ liệu thành các tập huấn luyện và kiểm tra
billsum = billsum.train_test_split(test_size=0.2)
billsum_test = load_dataset("billsum", split="ca_test")

len(billsum["train"]), len(billsum["test"]), len(billsum_test)

(989, 248, 3269)

In [ ]:
billsum["train"][0]

{'text': 'The people of the State of California do enact as follows:\n\n\nSECTION 1.\nSection 10203.4 of the Insurance Code is amended to read:\n10203.4.\n(a) Insurance under a group life insurance policy issued pursuant to Sections 10202, 10202.8, 10203, 10203.1, and 10203.7 may be extended to insure the dependents, or any class or classes thereof, of each insured employee who so elects, in amounts in accordance with some plan that precludes individual selection and that shall not be in excess of 100 percent of the insurance on the life of the insured employee. For dependent children over the age of majority, the group policyholder may elect coverage at age variations up to the limiting age.\n(b) “Dependent” includes the member’s spouse or a minor child, as well as a child older than the age of majority up to a maximum of 26 years of age, or any child over the age of majority who is both incapable of self-sustaining employment by reason of an intellectual disability or physical handic

### **Phương pháp học không giám sát(Textrank)**

TextRank – là mô hình xếp hạng dựa trên biểu đồ để xử lý văn bản, có thể được sử dụng để tìm các câu phù hợp nhất trong văn bản và cũng để tìm từ khóa.

https://web.eecs.umich.edu/~mihalcea/papers/mihalcea.emnlp04.pdf

#### **Xử lý dữ liệu**

In [ ]:
# Yêu cầu đầu vào của thuật toán là danh sách các câu
# Xử lý một văn bản thành danh sách các câu sau đó ánh xạ thành vector


#xử dụng pretrain bert-base
from transformers import BertTokenizer, BertModel
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained("bert-base-uncased")

def pre_docs(content):
    content = " ".join(content.split())
    content = content.replace("\n", " ")
    sentences = content.split(". ")
    return sentences

# corpus_embeddings
def embedding(sentences):
    embedding_list = []
    for sentence in sentences:
      encoded_input = tokenizer(sentence, return_tensors='pt', truncation=True)
      output = model(**encoded_input).pooler_output.tolist()[0]
      embedding_list.append(output)
    return embedding_list

#### **Xây dựng đồ thị**

In [ ]:
# Hàm để tính độ tương quan giữa câu
def calculate_cosine(u, v):
    result = 1 - spatial.distance.cosine(u, v)
    return result

# Xây dựng đồ thị câu với đỉnh là một câu và cạnh là độ tương quan giữa 2 câu
def build_graph(corpus_embeddings, ep):
    graph_len = len(corpus_embeddings)
    adj_matrix = np.full((graph_len, graph_len), 0.0)
    for i in range(0, graph_len):
        for j in range(0, graph_len):
            if i == j:
                continue
            cosine_sim = calculate_cosine(corpus_embeddings[i], corpus_embeddings[j])
            if cosine_sim > ep:
                adj_matrix[i][j] = cosine_sim
            else:
                adj_matrix[i][j] = 0.01
    return adj_matrix

A

# Hàm gọi thuật toán pagerank
def pagerank(a_matrix, eps=0.0001, d=0.85):
    p_matrix = np.ones(len(a_matrix)) / len(a_matrix)
    while True:
        new_p_matrix = np.ones(len(a_matrix)) * (1 - d) / len(
            a_matrix
        ) + d * a_matrix.T.dot(p_matrix)
        delta = abs(new_p_matrix - p_matrix).sum()
        if delta <= eps:
            return new_p_matrix
        p_matrix = new_p_matrix

In [ ]:
def textrank(corpus_embeddings, sentences, choose_sentences_len):
    if len(sentences) == 1:
        return sentences[0], sentences[0]
    graph_len = len(corpus_embeddings)
    adj_matrix = build_graph(corpus_embeddings, 0.0)

    for idx in range(len(adj_matrix)):
        adj_matrix[idx] /= adj_matrix[idx].sum()

    output = summary = ""

    P = pagerank(adj_matrix)

    _P = P.tolist()

    _P = np.argsort(_P)

    ranked_sentence_indexes = [
        item[0] for item in sorted(enumerate(P), key=lambda item: -item[1])
    ]
    selected_sentences = sorted(ranked_sentence_indexes[:choose_sentences_len])
    summary = itemgetter(*selected_sentences)(sentences)
    return " ".join(summary)

#### **Chạy thuật toán**

In [ ]:
# Học không giám sát nên không cần huấn luyện
ft_billsum_test = billsum_test[:10]
predict_list = []
gold_list = []
for i in range(len(ft_billsum_test)):
    content = ft_billsum_test["text"][i]
    sentences = pre_docs(content)
    corpus_embeddings = embedding(sentences)
    summary_result = textrank(corpus_embeddings, sentences, 3)
    predict_list.append(summary_result)
    gold_list.append(ft_billsum_test["summary"][i])

In [ ]:
rouge = evaluate.load('rouge')

results = rouge.compute(predictions=predict_list, references=gold_list)
results

{'rouge1': 0.23804323265797997,
 'rouge2': 0.08764810250181593,
 'rougeL': 0.19225603173407646,
 'rougeLsum': 0.19983178930983403}

In [ ]:
choose_case = 1
{
    "predict": predict_list[choose_case],
    "label": gold_list[choose_case]
}

{'predict': "That this Act may be cited as the ``Federal Forage Fee Act of 1993'' (5) Federal Advisory Committee Act (86 Stat The results of the updated survey shall be incorporated into the calculation of the Non Fee Cost Differential as they become available.",
 'label': 'Federal Forage Fee Act of 1993 - Subjects grazing operations on Federal land to applicable Federal, State, and local environmental and land use requirements. \nSets forth a forage fee formula for lands under the jurisdiction of the Department of Agriculture and the Department of the Interior.'}

### **Phương pháp học có giám sát(Deep learning method)**

#### **Tiền xử lý**

In [ ]:
#Tải lên bộ mã hoá được huấn luyện trước

from transformers import AutoTokenizer

PRETRAINED_MODEL = "t5-small"
tokenizer = AutoTokenizer.from_pretrained(PRETRAINED_MODEL)

In [ ]:
# Thêm dấu báo hiệu để giúp mô hình nhận biết nhiệm vụ
# Cắt bớt các chuỗi dài hơn độ dài tối đa
prefix = "summarize: "

# Hàm ánh xạ văn bản trở thành các vector biểu diễn
def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["text"]]
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True)

    labels = tokenizer(text_target=examples["summary"], max_length=128, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_billsum = billsum.map(preprocess_function, batched=True)
tokenized_billsum_test = billsum_test.map(preprocess_function, batched=True)

Map:   0%|          | 0/989 [00:00<?, ? examples/s]

Map:   0%|          | 0/248 [00:00<?, ? examples/s]

Map:   0%|          | 0/3269 [00:00<?, ? examples/s]

In [ ]:
#Tạo batch cho mô hình

from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=PRETRAINED_MODEL)

#### **Đánh giá**

Sử dụng Rouge score

In [ ]:
rouge = evaluate.load("rouge")

#Hàm dánh giá sau mỗi một epoch huấn luyện
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

#### **Xây dựng và huấn luyện mô hình**

In [ ]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(PRETRAINED_MODEL)

In [ ]:
# Tham số của mô hình
LR = 2e-5
TRAIN_BATCH_SIZE = 16
EVAL_BATCH_SIZE = 16
WEIGHT_DECAY = 0.01
EPOCHS = 3

# Tạo instance Trainer dựa vào các tham số đã định nghĩa

training_args = Seq2SeqTrainingArguments(
    output_dir="./",
    evaluation_strategy="epoch",
    learning_rate=LR,
    per_device_train_batch_size=TRAIN_BATCH_SIZE,
    per_device_eval_batch_size=EVAL_BATCH_SIZE,
    weight_decay=WEIGHT_DECAY,
    num_train_epochs=EPOCHS,
    predict_with_generate=True,
    fp16=True,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_billsum["train"],
    eval_dataset=tokenized_billsum["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,No log,2.809830,0.123100,0.033000,0.103100,0.102800,19.000000
2,No log,2.602016,0.129400,0.039600,0.108700,0.108700,19.000000
3,No log,2.562654,0.131100,0.040500,0.109600,0.109400,19.000000


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1260: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


TrainOutput(global_step=186, training_loss=3.174503121324765, metrics={'train_runtime': 220.7193, 'train_samples_per_second': 13.442, 'train_steps_per_second': 0.843, 'total_flos': 803118249934848.0, 'train_loss': 3.174503121324765, 'epoch': 3.0})

### **Infer**

In [ ]:
output_pair = trainer.predict(tokenized_billsum_test)

In [ ]:
output_pair.metrics

{'test_loss': 2.4660468101501465,
 'test_rouge1': 0.1095,
 'test_rouge2': 0.0419,
 'test_rougeL': 0.0912,
 'test_rougeLsum': 0.0912,
 'test_gen_len': 19.0,
 'test_runtime': 247.6423,
 'test_samples_per_second': 13.2,
 'test_steps_per_second': 0.828}

In [ ]:
choose_case = 1

{
  "predict": tokenizer.decode(output_pair.predictions[choose_case], skip_special_tokens=True),
  "label": billsum_test[choose_case]["summary"],
}

{'predict': 'SEC. 1. FINDINGS. (a) Findings.--Congress',
 'label': 'Federal Forage Fee Act of 1993 - Subjects grazing operations on Federal land to applicable Federal, State, and local environmental and land use requirements. \nSets forth a forage fee formula for lands under the jurisdiction of the Department of Agriculture and the Department of the Interior.'}

 **Bài tập 1:** Sử dụng bộ dữ liệu [OpenHust/vietnamese-summarization](https://huggingface.co/datasets/OpenHust/vietnamese-summarization) để chạy thử các phương pháp xây dựng mô hình tóm tắt ở trên. Và kiểm tra kết quả.

 **Bài tập 2:** Sử dụng các độ đo điểm tương đồng giữa 2 câu khác (VD: TF-IDF, W2V, ...) với thuật toán TextRank và so sánh kết quả.